In [ ]:
## Importar librerias 

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
## Cargamos los datos  

df = pd.read_excel('Datos_201901_202009.xlsx')
print(df.head())

In [ ]:
print(df.describe()) ## (!) campaña tomada como entero

In [ ]:
df = df.astype({'CAMPAÑA': str})

In [ ]:
print(df.info())

In [ ]:
print(df.isna().sum())

## Preparacion de los datos 

Se busca analizar las variables y encontrar corelaciones para armar el modelo

*La suma de los pedidos por grupo representa el 30% de las ventas totales

In [ ]:
sns.set(rc={"figure.figsize":(10, 8)}) 

In [ ]:
## Se graficaron los pedidos totales para los grupos del 1 al 10
for i in range(1,11):
    sns.scatterplot(data=df, x = f'GRUPO_{i}', y = 'PEDIDOS_TOTALES')

#### Correlación Pearson:

In [ ]:
sns.heatmap(df.corr(method='pearson'), annot=True)

Solo se consideraron las variables con un coeficiente de corelacion mayor a 0.3 (corelacion moderada). Por ende, las Xn para el modelo1 se consideraron como:

- Grupo 1 (corr = 0.53)
- Grupo 2 (corr = 0.45)
- Grupo 3 (corr = 0.31)

In [ ]:
## Se analizó los pedidos totales en funcion a cada campaña 
sns.set(rc={"figure.figsize":(28, 8)}) 
sns.scatterplot(data=df, x='CAMPAÑA', y= 'PEDIDOS_TOTALES',palette='Set2' ) 

---

In [ ]:
## Se analizaron los pedidos totales por grupo uno a uno para ver la distribucion de los datos 
sns.set(rc={"figure.figsize":(8, 8)}) 


sns.scatterplot(data=df, x='GRUPO_1', y= f'PEDIDOS_TOTALES',palette='Set2' ) #ok (corr = 0.53)

In [ ]:
sns.scatterplot(data=df, x='GRUPO_2', y= f'PEDIDOS_TOTALES',palette='Set2' ) #ok (corr = 0.45)

In [ ]:
sns.scatterplot(data=df, x='GRUPO_3', y= f'PEDIDOS_TOTALES',palette='Set2' ) #ok

In [ ]:
sns.scatterplot(data=df, x='GRUPO_4', y= f'PEDIDOS_TOTALES',palette='Set2' ) #no

In [ ]:
sns.scatterplot(data=df, x='GRUPO_5', y= f'PEDIDOS_TOTALES',palette='Set2' ) #no

In [ ]:
sns.scatterplot(data=df, x='GRUPO_6', y= f'PEDIDOS_TOTALES',palette='Set2' ) #no

In [ ]:
sns.scatterplot(data=df, x='GRUPO_7', y= f'PEDIDOS_TOTALES',palette='Set2' ) #ok // corr = 0.31

In [ ]:
sns.scatterplot(data=df, x='GRUPO_8', y= f'PEDIDOS_TOTALES',palette='Set2' ) #no

In [ ]:
sns.scatterplot(data=df, x='GRUPO_9', y= f'PEDIDOS_TOTALES',palette='Set2' ) #no

In [ ]:
sns.scatterplot(data=df, x='GRUPO_10', y= f'PEDIDOS_TOTALES',palette='Set2' ) #no // corr = 0.28

---

## Modelamiento de los datos


### Regresion Linea Multiple - Modelo 1

Y = Pedidos totales

Xn = Grupo1 , Grupo2, Grupo3 

Outcome: predecir los pedidos totales, en función a los pedidos de cada grupo n. 

Para este modelo se trabajó con los grupos 1,2,3 ya que su coeficiente de correlacion era mayor a 0.3

In [ ]:
# Librerias para el modelo de regreesion
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
for i in range(1,4):
    sns.scatterplot(data=df, x = f'GRUPO_{i}', y = 'PEDIDOS_TOTALES')

In [ ]:
## Se aplico un Modelo de Regresion Lineal para hallar los coeficientes de la ecuacion

X = df.iloc[:,0:3]
X


y = df['PEDIDOS_TOTALES']
X_train, X_test, y_train,y_test = train_test_split(X,y)

lin = LinearRegression()
lin.fit(X_train,y_train)
y_pred = lin.predict(X_train)
coef = lin.coef_
print(coef)

In [ ]:
## Ecuacion de la recta ## y = mx1x1 + ... m3x3 + b 

ecuacion = pd.DataFrame((zip)(X.columns,coef), columns =['grupo','coeficiente m'])
ecuacion = ecuacion.append({'grupo':'intercepto b','coeficiente m':lin.intercept_},ignore_index=True)
print(ecuacion)

ecuacion.to_csv('coeficientes.csv')

In [ ]:
## Graficamos los datos y la recta de regresion

for i in range(1,4):
    sns.scatterplot(data=df, x = f'GRUPO_{i}', y = 'PEDIDOS_TOTALES')

x_value = [x for x in range(0,9000)]
y_value = [x * coef[0] + coef[1] * 2 *x + coef[2] * 3 *x for x in x_value] + lin.intercept_

sns.lineplot(x_value,y_value)
plt.xlabel('Pedidos por grupo')
plt.ylabel('Pedidos totales')
plt.title('Ajuste de regresion lineal - Pedidos Totales vs G1,G2 y G3')


In [ ]:
g1 = int(input("Ingrese las ventas del Grupo 1: "))
g2 = int(input("Ingrese las ventas del Grupo 2: "))
g3 = int(input("Ingrese las ventas del Grupo 3: "))

ecuacion = round(coef[0]*g1 + coef[1]*g2 + coef[2]*g3 + lin.intercept_,2)
print("Los pedidos totales predichos segun el metodo de regresion lineal seran", ecuacion , "pedidos aproximadamente")

In [ ]:
## Evaluando el criterio R2 y MSE

print('Error cuadratico Medio: ' + str(mean_squared_error(y_train,y_pred)))
print('R2 score ' + str(r2_score(y_train,y_pred)))

In [ ]:
df["REGRESION_LINEAL"] = [(x) for x in y_value[:27]] ## igualando tamaño

In [ ]:
df

-----

### Regresion Lineal Simple - Modelo 2 

- Se modelaron las variables GRUPO individualmente para los 3 grupos
- Las variables de prueba y entrenamiento no fueron divididas 

In [ ]:
## Regresion lineal X = GRUPO_1

X1 = df['GRUPO_1'].values.reshape(-1,1)
y1 = df['PEDIDOS_TOTALES'].values.reshape(-1,1)
modeloG1 = LinearRegression()
modeloG1.fit(X1,y1)
y_pred1 = modeloG1.predict(X1)

modeloG1.coef_
modeloG1.intercept_


plt.scatter(X1,y1)
plt.plot(X1,y_pred1)
plt.ylabel('Ajuste de regresion lineal - Pedidos Totales vs G1')
plt.xlabel('PEDIDOS GRUPO_1')
plt.ylabel('PEDIDOS TOTALES')

print('Error cuadratico Medio: ' + str(mean_squared_error(y1,y_pred1)))
print('R2 score ' + str(r2_score(y1,y_pred1)))


In [ ]:
## Regresion lineal X = GRUPO_2

X2 = df['GRUPO_2'].values.reshape(-1,1)
y2 = df['PEDIDOS_TOTALES'].values.reshape(-1,1)
modeloG2 = LinearRegression()
modeloG2.fit(X2,y2)
y_pred2 = modeloG2.predict(X2)

modeloG2.coef_
modeloG2.intercept_


plt.scatter(X2,y2)
plt.plot(X2,y_pred2)
plt.ylabel('Ajuste de regresion lineal - Pedidos Totales vs G2')
plt.xlabel('PEDIDOS GRUPO_1')
plt.ylabel('PEDIDOS TOTALES')

print('Error cuadratico Medio: ' + str(mean_squared_error(y2,y_pred2)))
print('R2 score ' + str(r2_score(y2,y_pred2)))


In [ ]:
## Regresion lineal X = GRUPO_3

X3 = df['GRUPO_3'].values.reshape(-1,1)
y3 = df['PEDIDOS_TOTALES'].values.reshape(-1,1)
modeloG3 = LinearRegression()
modeloG3.fit(X3,y3)
y_pred3 = modeloG3.predict(X3)

modeloG3.coef_
modeloG3.intercept_


plt.scatter(X3,y3)
plt.plot(X3,y_pred3)
plt.ylabel('Ajuste de regresion lineal - Pedidos Totales vs G3')
plt.xlabel('PEDIDOS GRUPO_1')
plt.ylabel('PEDIDOS TOTALES')

print('Error cuadratico Medio: ' + str(mean_squared_error(y3,y_pred3)))
print('R2 score ' + str(r2_score(y3,y_pred3)))


In [ ]:
plt.plot(X1,y_pred1)
plt.plot(X2,y_pred2)
plt.plot(X3,y_pred3)

### Conclusiones y recomendaciones I



- Se concluye que este modelo de regresión lineal aplicado a los pedidos totales no ha sido adecuado para hacer predicciones acertadas, ya que su R2 tiende a ser inferior a 0.5, tomando como media valores de 0.3, llegando incluso a tener valores inferiores a 0.1 





- Se infiere que la relación entre los pedidos por grupo y los pedidos totales es No-Lineal.





- Las variables de respuesta elegidas, no han sido significativas para el modelo. Probablemente con una mayor cantidad de variables y datos, el modelo describiría mejor al comportamiento de los pedidos totales.

### Promedio Móvil:

Asumimos un comportamiento estacionario

Outcome: predecir los pedidos totales de la campaña n+1 basandonos en las n campañas pasadas. 

#### Variables:

Y = Pedidos totales


t = Campaña (201901 - 202009)


Periodo de medicion: inicio de una campaña. Es decir, 201901 -> 201902

In [ ]:
sns.scatterplot(data=df, x='CAMPAÑA', y= 'PEDIDOS_TOTALES',palette='Set2' ) 

In [ ]:
df["PEDIDOS_TOTALES"].plot(figsize =(10,10))

In [ ]:
df.rename({"CAMPAÑA" : "FECHA"}, axis=1,inplace=True)
df

In [ ]:
##df["FECHA"] = pd.to_datetime(df["FECHA"])
df = df.set_index("FECHA")
df

In [ ]:
df["PEDIDOS_TOTALES"].plot(figsize =(10,10))

In [ ]:
df["PEDIDOS_TOTALES"].plot(figsize =(10,10))
df['PEDIDOS_TOTALES'].rolling(window=3).mean().plot()

In [ ]:
df['PROMEDIO_MOVIL'] = df['PEDIDOS_TOTALES'].rolling(window=3).mean()


In [ ]:
mediciones = df['PEDIDOS_TOTALES'].count()

In [ ]:
pedidos_siguiente_campaña = df['PROMEDIO_MOVIL'].sum() / df['PROMEDIO_MOVIL'].count() ## Pronostico de los pedidos para la campaña n+1
print("Usando el metodo de Promedio Movil, los pedidos totales predichos para la medicion numero",mediciones+1, "seran", round(pedidos_siguiente_campaña,2),"pedidos aproximadamente")

### Conclusiones y recomendaciones II



- Se puede inferir que el comportamiento de la demanda es estacionario





- Este modelo tiene la limitación de que solo puede predecir un periodo mas (en este caso, la siguiente campaña)





- Para la realización del modelo solo se contó con 27 mediciones recientes, por lo que fue útil aplicar promedio móvil





- El promedio móvil pone énfasis en las ultimas mediciones, en caso se tengan mas datos históricos la capacidad de predecir los pedidos totales disminuiría ya que se podrían estar ignorando correlaciones y data significativa

